# Diccionarios (HashMap)

**Curso:** Estructuras de Datos (Ingeniería de Sistemas)

**Propósito:** Comprender y aplicar las ideas clave del tema, analizando complejidad y usos prácticos.

### Objetivos de aprendizaje
- Reconocer el modelo de datos y operaciones fundamentales.
- Analizar la complejidad temporal y espacial de las operaciones clave.
- Implementar y probar funciones en Python con casos de uso reales.
- Comparar ventajas y limitaciones frente a alternativas.


### Teoría esencial

Mapeo clave→valor basado en tabla hash. 
Operaciones promedio O(1) (peor caso O(n)). Claves deben ser hashables/inmutables.

### Guía de estudio
- Identifica operaciones fundamentales y su costo asintótico.
- Observa invariante(s) de la estructura (lo que siempre debe cumplirse).
- Relaciona la estructura con patrones de uso en software (colas de trabajo, caches, planificadores, etc.).

### Implementación base (desde el archivo `.py`)
El siguiente bloque contiene el código original.

In [ ]:
print('*** Sistema de Inventarios ***')

inventario = []

numero_productos = int(input('Cuantos productos deseas agregar al inventario? '))

for indice in range(numero_productos):
    print(f'Proporciona los valores del producto {indice + 1}')
    nombre = input('Nombre: ')
    precio = float(input('Precio: '))
    cantidad = int(input('Cantidad: '))
    # Creamos el diccionario con el detalle del producto
    producto = {'id': indice, 'nombre': nombre, 'precio': precio, 'cantidad': cantidad}
    # Agregamos el nuevo producto al inventario
    inventario.append(producto)

# Mostrar el inventario inicial
print(f'\nInventario inicial: {inventario}')

### Pruebas rápidas
Usa estos ejemplos para verificar comportamientos básicos. Ajusta según tus funciones.

In [ ]:
# Agrega aquí tus pruebas unitarias

### Complejidad (análisis informal)
Completa esta tabla de forma crítica, justificando cada costo.

| Operación | Mejor caso | Promedio | Peor caso | Nota |
|---|---|---|---|---|
| op1 | O( ) | O( ) | O( ) |  |
| op2 | O( ) | O( ) | O( ) |  |
| op3 | O( ) | O( ) | O( ) |  |

### Ejercicios propuestos
1. Implementa pruebas unitarias con `pytest` para al menos 5 funciones/operaciones.
2. Mide empíricamente tiempos (con `timeit`) al variar n, y compara con el análisis teórico.
3. Extiende la implementación para cubrir un caso límite (overflow, colisión, degeneración, etc.).
4. Escribe una breve reflexión: ¿en qué contextos reales usarías esta estructura sobre sus alternativas?

### Referencias rápidas
- Cormen, Leiserson, Rivest, Stein. *Introduction to Algorithms* (CLRS).
- Sedgewick & Wayne. *Algorithms*.
- Documentación oficial de Python: `collections`, `heapq`, `bisect`, `array`.

*Fuente de código:* `14-Solucion-SistemaInventario-UP.py`  — Generado automáticamente el 2025-09-08 22:41.